# Web Scraping
* 網站使用 HTML 做為預設的文件格式, 由於 HTML 有規範與標準, 瀏覽器可以依據規範來顯示畫面
* 有時候 HTML 文件沒有依照規範來編排, 這種狀況被稱為 tag soup
* 使用 requests 模組擷取 HTML 文件, 再透過 BeautifulSoup 模組分析 tag 內容, 是常見的處理方式

# requests 模組
* 存取網站檔案的優秀工具
* 網頁檔案常見的存取方式是 Get 和 Post 模式

In [ ]:
# 載入模組
import requests

In [ ]:
# 回傳 Response 物件
req = requests.get("https://api.github.com/events")

In [ ]:
# 顯示 Response 內容
req.text

In [ ]:
# 在網址輸入參數
payload = {'key1': 'value1', 'key2': 'value2'}
req = requests.get('http://httpbin.org/get', params=payload)

In [ ]:
print(req.url)

In [ ]:
# 參數可以是 List of Strings
payload = {'key1': 'value1', 'key2': ['value2', 'value3']}
req = requests.get('http://httpbin.org/get', params=payload)

In [ ]:
print(req.url)

# BeautifulSoup 模組
* 用來分析存取 HTML 文件的工具
* 輸入的資料預設會使用 Unicode 編碼, 輸出的資料預設使用 UTF-8 編碼

In [ ]:
from bs4 import BeautifulSoup
import requests

url = ""
req = requests.get(url)
data = req.text
soup = BeautifulSoup(data)

for link in soup.find_all('a'):
    print(link.get('href'))